In [22]:
import pandas as pd
import requests
import bs4
import os
import re
import time

In [48]:
file_list = []
for i in os.listdir():
    if "k.csv"in i:
        file_list.append(i)
print(file_list)

['book1-100k.csv', 'book1000k-1100k.csv', 'book100k-200k.csv', 'book1100k-1200k.csv', 'book1200k-1300k.csv', 'book1300k-1400k.csv', 'book1400k-1500k.csv', 'book1500k-1600k.csv', 'book1600k-1700k.csv', 'book1700k-1800k.csv', 'book1800k-1900k.csv', 'book1900k-2000k.csv', 'book2000k-3000k.csv', 'book200k-300k.csv', 'book3000k-4000k.csv', 'book300k-400k.csv', 'book4000k-5000k.csv', 'book400k-500k.csv', 'book500k-600k.csv', 'book600k-700k.csv', 'book700k-800k.csv', 'book800k-900k.csv', 'book900k-1000k.csv']


In [11]:
# get original columns for append
init_column = pd.read_csv(file_list[0]).columns
# create a big df for crawler and id-join
final_df = pd.DataFrame(columns=init_column)
for i in file_list:
    final_df = final_df.append(pd.read_csv(i))

In [41]:
final_df = final_df.reset_index(drop= True)
final_df = final_df.drop_duplicates()
final_df

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,...,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3,Description,Count of text reviews,PagesNumber
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,...,eng,J.K. Rowling,4.57,2:25317,5:1546466,NaN,3:159960,NaN,NaN,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,...,eng,J.K. Rowling,4.50,2:37005,5:1493113,0439358078,3:211781,NaN,NaN,NaN
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,1:108202,309,4:1513191,total:6587388,1,11,Scholastic Inc,75911,...,eng,J.K. Rowling,4.47,2:130310,5:4268227,NaN,3:567458,NaN,NaN,NaN
3,4,Harry Potter and the Chamber of Secrets (Harry...,1:11896,352,4:706082,total:2560657,1,11,Scholastic,244,...,eng,J.K. Rowling,4.42,2:49353,5:1504505,0439554896,3:288821,NaN,NaN,NaN
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,1:10128,435,4:630534,total:2610317,1,5,Scholastic Inc.,37093,...,eng,J.K. Rowling,4.57,2:24849,5:1749958,043965548X,3:194848,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850305,999991,Gendered Talk at Work: Constructing Gender Ide...,1:0,251,4:3,total:25,21,7,Wiley-Blackwell,0,...,NaN,Janet Holmes,4.28,2:1,5:15,1405117591,3:6,<i>Gendered Talk at Work</i> examines how wome...,0.0,NaN
1850306,999992,Good Will Toward Men: Women Talk Candidly abou...,1:0,231,4:0,total:0,1,2,St. Martin's Press,0,...,NaN,Jack Kammer,0.00,2:0,5:0,0312104715,3:0,In interviews with gender issues journalist Ja...,0.0,NaN
1850307,999993,Would You Rather . . . ?: The Outrageous Book ...,1:2,400,4:35,total:121,1,10,Workman Publishing Company,14,...,NaN,Randy Horn,3.81,2:11,5:39,0761124209,3:34,<i>Would You Rather...?</i> takes the idea of ...,14.0,NaN
1850308,999997,Addictions Treatment for Older Adults: Evaluat...,1:0,258,4:0,total:0,2,2,Routledge,0,...,NaN,Kathryn Graham,0.00,2:0,5:0,1560248572,3:0,Here is a detailed description of an innovativ...,0.0,NaN


In [92]:
Books = pd.read_csv("Books.csv")

C:\Users\Austin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [111]:
Genre_df = final_df.copy()
# drop NA value in ISBN
Genre_df= Genre_df.loc[Genre_df.ISBN.dropna().index,:]
# filter by ISBN
Genre_df = Genre_df.merge(Books,on = "ISBN")
# generate empty columns
Genre_df.loc[:,"Genre"] = ""
Genre_df.loc[:,"Genre_user_amount"] = ""

In [86]:
# Crawler
def Crawler(ID = 1):
    genre_string = ""
    genre_users_string = ""
    response  = requests.get(f"https://www.goodreads.com/book/show/{ID}")
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    genre_list = soup.find_all("a", {"class": "bookPageGenreLink"})
    genre_amount = soup.find_all("div", {"class": "bookPageGenreLink"})
    for i in range(min(len(genre_list),len(genre_amount))):
        genre_string += genre_list[i].text
        m = re.search(r"(\d{0,3},?\d{0,3},?\d{0,3}) users",genre_amount[i].text)
        genre_users_string +=  m.group(1)

        genre_string += "|"
        genre_users_string += "|"
    
    return genre_string, genre_users_string


In [ ]:
for k,i in enumerate(Genre_df.Id):
    Genre_df.iloc[k,-2], Genre_df.iloc[k,-1] = Crawler(ID = i)
    print(f"Book number:{k} out of 141480 ({k/141810*100:.3f}%)")

Book number:0 out of 141480 (0.000%)
Book number:1 out of 141480 (0.001%)
Book number:2 out of 141480 (0.001%)
Book number:3 out of 141480 (0.002%)
Book number:4 out of 141480 (0.003%)
Book number:5 out of 141480 (0.004%)
Book number:6 out of 141480 (0.004%)
Book number:7 out of 141480 (0.005%)
Book number:8 out of 141480 (0.006%)
Book number:9 out of 141480 (0.006%)
Book number:10 out of 141480 (0.007%)
Book number:11 out of 141480 (0.008%)
Book number:12 out of 141480 (0.008%)
Book number:13 out of 141480 (0.009%)
Book number:14 out of 141480 (0.010%)
Book number:15 out of 141480 (0.011%)
Book number:16 out of 141480 (0.011%)
Book number:17 out of 141480 (0.012%)
Book number:18 out of 141480 (0.013%)
Book number:19 out of 141480 (0.013%)
Book number:20 out of 141480 (0.014%)
Book number:21 out of 141480 (0.015%)
Book number:22 out of 141480 (0.016%)
Book number:23 out of 141480 (0.016%)
Book number:24 out of 141480 (0.017%)
Book number:25 out of 141480 (0.018%)
Book number:26 out of 

Book number:213 out of 141480 (0.150%)
Book number:214 out of 141480 (0.151%)
Book number:215 out of 141480 (0.152%)
Book number:216 out of 141480 (0.152%)
Book number:217 out of 141480 (0.153%)
Book number:218 out of 141480 (0.154%)
Book number:219 out of 141480 (0.154%)
Book number:220 out of 141480 (0.155%)
Book number:221 out of 141480 (0.156%)
Book number:222 out of 141480 (0.157%)
Book number:223 out of 141480 (0.157%)
Book number:224 out of 141480 (0.158%)
Book number:225 out of 141480 (0.159%)
Book number:226 out of 141480 (0.159%)
Book number:227 out of 141480 (0.160%)
Book number:228 out of 141480 (0.161%)
Book number:229 out of 141480 (0.161%)
Book number:230 out of 141480 (0.162%)
Book number:231 out of 141480 (0.163%)
Book number:232 out of 141480 (0.164%)
Book number:233 out of 141480 (0.164%)
Book number:234 out of 141480 (0.165%)
Book number:235 out of 141480 (0.166%)
Book number:236 out of 141480 (0.166%)
Book number:237 out of 141480 (0.167%)
Book number:238 out of 14

Book number:424 out of 141480 (0.299%)
Book number:425 out of 141480 (0.300%)
Book number:426 out of 141480 (0.300%)
Book number:427 out of 141480 (0.301%)
Book number:428 out of 141480 (0.302%)
Book number:429 out of 141480 (0.303%)
Book number:430 out of 141480 (0.303%)
Book number:431 out of 141480 (0.304%)
Book number:432 out of 141480 (0.305%)
Book number:433 out of 141480 (0.305%)
Book number:434 out of 141480 (0.306%)
Book number:435 out of 141480 (0.307%)
Book number:436 out of 141480 (0.307%)
Book number:437 out of 141480 (0.308%)
Book number:438 out of 141480 (0.309%)
Book number:439 out of 141480 (0.310%)
Book number:440 out of 141480 (0.310%)
Book number:441 out of 141480 (0.311%)
Book number:442 out of 141480 (0.312%)
Book number:443 out of 141480 (0.312%)
Book number:444 out of 141480 (0.313%)
Book number:445 out of 141480 (0.314%)
Book number:446 out of 141480 (0.315%)
Book number:447 out of 141480 (0.315%)
Book number:448 out of 141480 (0.316%)
Book number:449 out of 14

Book number:635 out of 141480 (0.448%)
Book number:636 out of 141480 (0.448%)
Book number:637 out of 141480 (0.449%)
Book number:638 out of 141480 (0.450%)
Book number:639 out of 141480 (0.451%)
Book number:640 out of 141480 (0.451%)
Book number:641 out of 141480 (0.452%)
Book number:642 out of 141480 (0.453%)
Book number:643 out of 141480 (0.453%)
Book number:644 out of 141480 (0.454%)
Book number:645 out of 141480 (0.455%)
Book number:646 out of 141480 (0.456%)
Book number:647 out of 141480 (0.456%)
Book number:648 out of 141480 (0.457%)
Book number:649 out of 141480 (0.458%)
Book number:650 out of 141480 (0.458%)
Book number:651 out of 141480 (0.459%)
Book number:652 out of 141480 (0.460%)
Book number:653 out of 141480 (0.460%)
Book number:654 out of 141480 (0.461%)
Book number:655 out of 141480 (0.462%)
Book number:656 out of 141480 (0.463%)
Book number:657 out of 141480 (0.463%)
Book number:658 out of 141480 (0.464%)
Book number:659 out of 141480 (0.465%)
Book number:660 out of 14

Book number:846 out of 141480 (0.597%)
Book number:847 out of 141480 (0.597%)
Book number:848 out of 141480 (0.598%)
Book number:849 out of 141480 (0.599%)
Book number:850 out of 141480 (0.599%)
Book number:851 out of 141480 (0.600%)
Book number:852 out of 141480 (0.601%)
Book number:853 out of 141480 (0.602%)
Book number:854 out of 141480 (0.602%)
Book number:855 out of 141480 (0.603%)
Book number:856 out of 141480 (0.604%)
Book number:857 out of 141480 (0.604%)
Book number:858 out of 141480 (0.605%)
Book number:859 out of 141480 (0.606%)
Book number:860 out of 141480 (0.606%)
Book number:861 out of 141480 (0.607%)
Book number:862 out of 141480 (0.608%)
Book number:863 out of 141480 (0.609%)
Book number:864 out of 141480 (0.609%)
Book number:865 out of 141480 (0.610%)
Book number:866 out of 141480 (0.611%)
Book number:867 out of 141480 (0.611%)
Book number:868 out of 141480 (0.612%)
Book number:869 out of 141480 (0.613%)
Book number:870 out of 141480 (0.613%)
Book number:871 out of 14

...

Book number:1875 out of 141480 (1.322%)
Book number:1876 out of 141480 (1.323%)
Book number:1877 out of 141480 (1.324%)
Book number:1878 out of 141480 (1.324%)
Book number:1879 out of 141480 (1.325%)
Book number:1880 out of 141480 (1.326%)
Book number:1881 out of 141480 (1.326%)
Book number:1882 out of 141480 (1.327%)
Book number:1883 out of 141480 (1.328%)
Book number:1884 out of 141480 (1.329%)
Book number:1885 out of 141480 (1.329%)
Book number:1886 out of 141480 (1.330%)
Book number:1887 out of 141480 (1.331%)
Book number:1888 out of 141480 (1.331%)
Book number:1889 out of 141480 (1.332%)
Book number:1890 out of 141480 (1.333%)
Book number:1891 out of 141480 (1.333%)
Book number:1892 out of 141480 (1.334%)
Book number:1893 out of 141480 (1.335%)
Book number:1894 out of 141480 (1.336%)
Book number:1895 out of 141480 (1.336%)
Book number:1896 out of 141480 (1.337%)
Book number:1897 out of 141480 (1.338%)
Book number:1898 out of 141480 (1.338%)
Book number:1899 out of 141480 (1.339%)


Book number:2080 out of 141480 (1.467%)
Book number:2081 out of 141480 (1.467%)
Book number:2082 out of 141480 (1.468%)
Book number:2083 out of 141480 (1.469%)
Book number:2084 out of 141480 (1.470%)
Book number:2085 out of 141480 (1.470%)
Book number:2086 out of 141480 (1.471%)
Book number:2087 out of 141480 (1.472%)
Book number:2088 out of 141480 (1.472%)
Book number:2089 out of 141480 (1.473%)
Book number:2090 out of 141480 (1.474%)
Book number:2091 out of 141480 (1.475%)
Book number:2092 out of 141480 (1.475%)
Book number:2093 out of 141480 (1.476%)
Book number:2094 out of 141480 (1.477%)
Book number:2095 out of 141480 (1.477%)
Book number:2096 out of 141480 (1.478%)
Book number:2097 out of 141480 (1.479%)
Book number:2098 out of 141480 (1.479%)
Book number:2099 out of 141480 (1.480%)
Book number:2100 out of 141480 (1.481%)
Book number:2101 out of 141480 (1.482%)
Book number:2102 out of 141480 (1.482%)
Book number:2103 out of 141480 (1.483%)
Book number:2104 out of 141480 (1.484%)


Book number:2285 out of 141480 (1.611%)
Book number:2286 out of 141480 (1.612%)
Book number:2287 out of 141480 (1.613%)
Book number:2288 out of 141480 (1.613%)
Book number:2289 out of 141480 (1.614%)
Book number:2290 out of 141480 (1.615%)
Book number:2291 out of 141480 (1.616%)
Book number:2292 out of 141480 (1.616%)
Book number:2293 out of 141480 (1.617%)
Book number:2294 out of 141480 (1.618%)
Book number:2295 out of 141480 (1.618%)
Book number:2296 out of 141480 (1.619%)
Book number:2297 out of 141480 (1.620%)
Book number:2298 out of 141480 (1.620%)
Book number:2299 out of 141480 (1.621%)
Book number:2300 out of 141480 (1.622%)
Book number:2301 out of 141480 (1.623%)
Book number:2302 out of 141480 (1.623%)
Book number:2303 out of 141480 (1.624%)
Book number:2304 out of 141480 (1.625%)
Book number:2305 out of 141480 (1.625%)
Book number:2306 out of 141480 (1.626%)
Book number:2307 out of 141480 (1.627%)
Book number:2308 out of 141480 (1.628%)
Book number:2309 out of 141480 (1.628%)


Book number:2490 out of 141480 (1.756%)
Book number:2491 out of 141480 (1.757%)
Book number:2492 out of 141480 (1.757%)
Book number:2493 out of 141480 (1.758%)
Book number:2494 out of 141480 (1.759%)
Book number:2495 out of 141480 (1.759%)
Book number:2496 out of 141480 (1.760%)
Book number:2497 out of 141480 (1.761%)
Book number:2498 out of 141480 (1.762%)
Book number:2499 out of 141480 (1.762%)
Book number:2500 out of 141480 (1.763%)
Book number:2501 out of 141480 (1.764%)
Book number:2502 out of 141480 (1.764%)
Book number:2503 out of 141480 (1.765%)
Book number:2504 out of 141480 (1.766%)
Book number:2505 out of 141480 (1.766%)
Book number:2506 out of 141480 (1.767%)
Book number:2507 out of 141480 (1.768%)
Book number:2508 out of 141480 (1.769%)
Book number:2509 out of 141480 (1.769%)
Book number:2510 out of 141480 (1.770%)
Book number:2511 out of 141480 (1.771%)
Book number:2512 out of 141480 (1.771%)
Book number:2513 out of 141480 (1.772%)
Book number:2514 out of 141480 (1.773%)


Book number:2695 out of 141480 (1.900%)
Book number:2696 out of 141480 (1.901%)
Book number:2697 out of 141480 (1.902%)
Book number:2698 out of 141480 (1.903%)
Book number:2699 out of 141480 (1.903%)
Book number:2700 out of 141480 (1.904%)
Book number:2701 out of 141480 (1.905%)
Book number:2702 out of 141480 (1.905%)
Book number:2703 out of 141480 (1.906%)
Book number:2704 out of 141480 (1.907%)
Book number:2705 out of 141480 (1.907%)
Book number:2706 out of 141480 (1.908%)
Book number:2707 out of 141480 (1.909%)
Book number:2708 out of 141480 (1.910%)
Book number:2709 out of 141480 (1.910%)
Book number:2710 out of 141480 (1.911%)
Book number:2711 out of 141480 (1.912%)
Book number:2712 out of 141480 (1.912%)
Book number:2713 out of 141480 (1.913%)
Book number:2714 out of 141480 (1.914%)
Book number:2715 out of 141480 (1.915%)
Book number:2716 out of 141480 (1.915%)
Book number:2717 out of 141480 (1.916%)
Book number:2718 out of 141480 (1.917%)
Book number:2719 out of 141480 (1.917%)


In [134]:
# sample_df = Genre_df[Genre_df.Genre!=""]

In [136]:
# sample_df.to_csv("Books_sample.csv")

In [135]:
# sample_df.tail(5)

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher_x,CountsOfReview,...,PagesNumber,Book-Title,Book-Author,Year-Of-Publication,Publisher_y,Image-URL-S,Image-URL-M,Image-URL-L,Genre,Genre_user_amount
1319,10217,Death in Kashmir,1:19,256,4:702,total:1717,1,12,St. Martins Press-3PL,106,...,NaN,Death in Kashmir: A Mystery,M. M. Kaye,2000,St. Martin's Minotaur,http://images.amazon.com/images/P/0312263104.0...,http://images.amazon.com/images/P/0312263104.0...,http://images.amazon.com/images/P/0312263104.0...,Mystery|Cultural|India|Historical|Historical F...,168|33|32|25|19|14|14|12|10|9|
1374,10656,Next: The Future Just Happened,1:18,240,4:628,total:1581,17,5,W. W. Norton Company,106,...,NaN,Next: The Future Just Happened,Michael Lewis,2002,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393323528.0...,http://images.amazon.com/images/P/0393323528.0...,http://images.amazon.com/images/P/0393323528.0...,Nonfiction|Business|Science|Technology|Compute...,62|42|10|9|9|8|8|5|5|4|
1394,10801,The Big Rock Candy Mountain,1:69,563,4:2660,total:6753,1,3,Penguin,672,...,NaN,The Big Rock Candy Mountain (Contemporary Amer...,Wallace Stegner,1995,Penguin Books,http://images.amazon.com/images/P/0140139397.0...,http://images.amazon.com/images/P/0140139397.0...,http://images.amazon.com/images/P/0140139397.0...,Fiction|Historical|Historical Fiction|Classics...,323|97|62|25|25|24|17|17|17|14|
1411,10890,Traveling Mercies: Some Thoughts on Faith,1:521,275,4:15301,total:41458,15,2,Anchor,2309,...,NaN,Traveling Mercies: Some Thoughts on Faith,Anne Lamott,2000,Anchor Books/Doubleday,http://images.amazon.com/images/P/0385496095.0...,http://images.amazon.com/images/P/0385496095.0...,http://images.amazon.com/images/P/0385496095.0...,Nonfiction|Autobiography|Memoir|Spirituality|R...,655|530|366|339|211|163|133|103|70|59|
1420,10972,"Omega Path (Outlanders, #4)",1:1,352,4:27,total:89,23,1,Gold Eagle,2,...,NaN,"Omega Path (Outlanders #4) (Outlanders , No 360)",James Axler,1998,Gold Eagle,http://images.amazon.com/images/P/0373638175.0...,http://images.amazon.com/images/P/0373638175.0...,http://images.amazon.com/images/P/0373638175.0...,Science Fiction|Apocalyptic|,3|3|


In [ ]:
Genre_df.to_csv("All_Bookdata.csv")